# Linear Regression Implementation (Single variable)

In [1]:
import numpy as np
import pandas as pd

# Use plotly as it is an interaction plot
import plotly.express as px
# sub plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Load data

In [2]:
X_train, y_train = np.array([4.529, 6.552, 8.725]), np.array([7, 31.5, 50])

# Linear regression

##### Formula

$ f_{w,b}(x^{(i)}) = wx^{(i)} + b $

In [41]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=X_train,
        y=y_train,
        mode='markers',
    ))

# include shapes in layout
fig.update_layout(height=400, width=600, title_text="Target")
fig.update_xaxes(title_text='X')
fig.update_yaxes(title_text='y')
fig.show()


In [4]:
# Loop implementation
def compute_linear_regression(X, w, b):
    """
    Computes the prediction of a linear model
    Input X compute with parameter w,b
    return prediction value
    
    Args:
        X (ndarray (m,n)): Data, m examples 
        w,b (scalar)    : model parameters  
    Returns
        f_wb (ndarray (m,)): predict values
    """
    
    m = X.shape[0]
    f_wb = np.zeros(m)
    
    for i in range(m):
        f_wb[i] = w * X[i] + b
        
    return f_wb

In [5]:
w_init = 10
b_init = -40
f_wb = compute_linear_regression(X=X_train, w=w_init, b=b_init)
f_wb

array([ 5.29, 25.52, 47.25])

In [6]:
# Vectorized implementation
def compute_linear_regression_v(X, w, b):
    f_wb = w * X + b
    return f_wb

In [7]:
f_wb = compute_linear_regression_v(X=X_train, w=w_init , b=b_init)
f_wb

array([ 5.29, 25.52, 47.25])

In [40]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=X_train,
        y=y_train,
        mode='markers',
        name='Target'
    ))

fig.add_trace(
    go.Line(
        x=X_train,
        y=f_wb,
        name='Predict'
    ))

shapes = []

m = X_train.shape[0]

for i in range(m):
    shapes.append(
        go.layout.Shape(
            type="line",
            x0=X_train[i],
            y0=f_wb[i],
            x1=X_train[i],
            y1=y_train[i],
            line=dict(
                #color=np.random.choice(colors,1)[0],
                color = 'black',
                width=1),
            opacity=1,
            layer='above'
            )
        )

# include shapes in layout
fig.update_layout(shapes=shapes)
fig.update_xaxes(title_text='X')
fig.update_yaxes(title_text='y')
fig.update_layout(height=400, width=600, title_text="Linear regression")
fig.show()

# Cost function

$ J(w, b) = \frac{1}{2m} \sum_{i=1}^{m} (f_{w,b}(x^{(i)}) - y^{(i)})^2 $

In [9]:
def compute_cost(X, y, w, b):
    """
    Computes the prediction of a linear model
    Args:
        X (ndarray (m,n)) : Data, m examples
        y (ndarray (m,))  : target values
        w,b (scalar)      : model parameters  
    Returns
        cost (int)        : cost error value
    """
    
    m = X.shape[0]    

    f_wb = compute_linear_regression(X, w, b)
    
    error_sum = 0
    
    for i in range(m):
        error_sum += (f_wb[i] - y[i]) ** 2
        cost = error_sum / (2 * m)
    return cost

In [10]:
def compute_cost_v(X, y, w, b):
    f_wb = compute_linear_regression_v(X, w, b)
    cost = ((f_wb - y) ** 2).mean() / 2
    return cost

In [11]:
compute_cost(X_train, y_train, w=w_init, b=b_init)

7.707833333333341

Cost is 7.7, Let's see how our prediction do.

Gradeint Descent

$ \{ $
    
$ w^{(i)}_j := w^{(i)}_j - \alpha \frac{\sigma}{\sigma w}J(w, b)x^{(i)}_j $

$ b^{(i)} := b^{(i)} - \alpha \frac{\sigma}{\sigma w}J(w, b) $

$ \} {stimulous update} $


$ \frac{\sigma}{\sigma w}J(w, b) = \frac{1}{m} \sum_{i=1}^{m} (h_{\theta}(x^{(i)}) - y^{(i)}) $

In [12]:
def gradient_function(X, y, w, b):
    """
    Computes the prediction of a linear model
    Args:
        X (np.array (m,n))      : Data, m examples
        y (np.array (m,))       : target values
        w,b (scalar)            : init model parameters  
    Returns
        dj_dw, dj_db (scalar)   : tuned model parameters  
    """
    
    m = X.shape[0]
    dj_dw = 0
    dj_db = 0

    for i in range(m):
        f_wb = compute_linear_regression(X, w, b)
        dj_dw += (f_wb[i] - y[i]) * X[i]
        dj_db += f_wb[i] - y[i]
    dj_dw = dj_dw / m
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

In [13]:
a,b = gradient_function(X_train, y_train, w=10.5, b=-40)
print(f'{a} {b}')

-0.37873833333336293 -0.1790000000000044


In [14]:
def gradient_function_v(X, y, w, b):
    
    m = X.shape[0]
    
    dj_dw = 0
    dj_db = 0

    f_wb = compute_linear_regression_v(X, w, b)
    error = f_wb - y
    dj_dw = error.T.dot(X)
    dj_db = sum(error)
    dj_dw = dj_dw / m
    dj_db = dj_db / m
        
    return dj_dw, dj_db

In [15]:
a,b = gradient_function_v(X_train, y_train, w=10.5, b=-40)
print(f'{a} {b}')

-0.37873833333336226 -0.1790000000000044


In [16]:
def gradient_descent(X, y, w, b, alpha, num_iters, cost_function, gradient_function):
    
    j_hist = []
    p_hist = []

    for i in range(num_iters):
        
        dj_dw, dj_db = gradient_function(X, y, w, b)
        
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
    
        j_hist.append(cost_function(X, y, w, b))
        p_hist.append([w,b])
        
    return w, b, j_hist, p_hist

In [17]:
w, b, j_hist, p_hist = gradient_descent(X_train, y_train, w=0, b=0 ,alpha=0.03, num_iters=5000, cost_function=compute_cost, gradient_function=gradient_function)

In [18]:
j_hist

[145.86888810978954,
 65.91738456429117,
 51.41470044196171,
 48.66790959063007,
 48.03313944852263,
 47.77823700142236,
 47.592085587959254,
 47.41880312007262,
 47.24835059854892,
 47.07892298035412,
 46.910194143355056,
 46.74210360382158,
 46.574638596829544,
 46.4077949353542,
 46.2415699800846,
 46.07596137669481,
 45.91096682901729,
 45.746584058276305,
 45.582810795740194,
 45.419644781375816,
 45.257083763581306,
 45.095125499112676,
 44.93376775304529,
 44.77300829874156,
 44.61284491781993,
 44.453275400124426,
 44.29429754369395,
 44.13590915473202,
 43.97810804757634,
 43.82089204466886,
 43.664258976525524,
 43.50820668170644,
 43.35273300678608,
 43.19783580632343,
 43.043512942832514,
 42.88976228675286,
 42.73658171642011,
 42.5839691180367,
 42.43192238564276,
 42.28043942108699,
 42.12951813399782,
 41.979156441754355,
 41.82935226945791,
 41.68010354990307,
 41.5314082235495,
 41.383264238493176,
 41.23566955043844,
 41.0886221226694,
 40.94211992602212,
 40.7961609

In [19]:
print(f'result cost:{min(j_hist)} with parameter w:{w} b:{b}')

result cost:1.5773578196558418 with parameter w:10.22595033983032 b:-38.01150247588224


In [20]:
compute_cost(X_train, y_train, w=w, b=b)

1.5773578196558418

#### Let's plot Cost function

In [37]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(j_hist))],
        y=j_hist,
        mode='lines+markers',
        name='Cost'
    ))

fig.update_xaxes(title_text="Iteration")
fig.update_yaxes(title_text="Cost")
fig.update_layout(height=400, width=600, title_text="Cost Function")
fig.show()

In [22]:
print(f'result cost:{min(j_hist)} with parameter w:{w} b:{b}')

result cost:1.5773578196558418 with parameter w:10.22595033983032 b:-38.01150247588224


Result: Best fit value is as shown above

Not bad


Let's check our result with parameter w,b

and plot to see how the model fit the targets

In [23]:
compute_cost(X_train, y_train, w=w, b=b)

1.5773578196558418

Our score is 1.57, much better than 7.70 before our compute is work!

In [25]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        x=X_train,
        y=y_train,
        mode='markers',
        name='Target'
    ))

fig.add_trace(
    go.Line(
        x=X_train,
        y=f_wb,
        name='Predict'
    ))

shapes = []

m = X_train.shape[0]

for i in range(m):
    shapes.append(
        go.layout.Shape(
            type="line",
            x0=X_train[i],
            y0=f_wb[i],
            x1=X_train[i],
            y1=y_train[i],
            line=dict(
                #color=np.random.choice(colors,1)[0],
                color = 'black',
                width=1),
            opacity=1,
            layer='above'
            )
        )

# include shapes in layout
fig.update_layout(shapes=shapes)
fig.update_layout(height=400, width=600, title_text="Linear regression")
fig.show()

/Users/krit/homebrew/Caskroom/miniforge/base/envs/mlp-env/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


